# Dynamics via acceleration maps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from analysis import analysis

from importlib import reload

L_box = 50
mu_factor = 6  # conversion to microns
min_factor = 8  # conversion to minutes


In [ ]:
import yaml


def _get_mp_type(yfile):
    with open(yfile) as f:
        return yaml.safe_load(f)["substrate"]["kind"]


def _make_title(df):
    tbl = {
        "gamma": r"$\gamma$",
        "R_eq": r"$R_{eq}$",
        "mag_std": r"$\sigma_{MVG}$",
        "add_rate": r"$\tau_{MVG}$",
        "gid": "ID",
    }
    d = dict(df.iloc[0][3:8])
    title = ""
    for key, val in d.items():
        title += tbl[key] + " = " + f"{val}" + "\n"
    return title

def linear_init_pts(xmin, xmax, vmin, vmax, n_pts, s=1, basin_only=False):
    def _around_basin():
        d = 10
        x1 = np.linspace(xmin - d, xmin + d, n_pts)
        x2 = np.linspace(xmax - d, xmax + d, n_pts)
        x = np.append(x1, x2)
        y = s * (vmax - vmin) / (xmax - xmin) * (x - xmin) + vmin
        return np.hstack([x.reshape(-1, 1), y.reshape(-1, 1)])

    if basin_only:
        return _around_basin()

    x = np.linspace(xmin, xmax, n_pts)
    y = s * (vmax - vmin) / (xmax - xmin) * (x - xmin) + vmin
    return np.hstack([x.reshape(-1, 1), y.reshape(-1, 1)])


def F_init_pts(F, bounds):
    y_indx, x_indx = np.where(~np.isnan(F))
    xmin, xmax, vmin, vmax, nbins = bounds

    yy = vmin + y_indx * (vmax - vmin) / nbins
    xx = xmin + x_indx * (xmax - xmin) / nbins
    return np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1)])


def get_xv_traj_for_gid(gid):
    df_gid_singleRunLong = pd.read_pickle(
        f"../_server/sim_data/corners_only_2/single_run_long/pkls/fulltake_gid{gid}.pkl"
    )
    df_gid_singleRunLong.x *= mu_factor
    df_gid_singleRunLong.y *= mu_factor

    singleRun_xva = analysis.calc_v_a_from_position(
        df_gid_singleRunLong.x, df_gid_singleRunLong["time[hr]"]
    )
    return singleRun_xva[["x", "v"]].to_numpy()


#### Load grid by grid
- Load `fulltake_gid*.pkl`, which is all of the runs for a config

In [ ]:
from glob import glob

files = glob("../_server/sim_data/corners_only_2/pkls/fulltake_gid*.pkl")
# files_2 = glob("../_server/sim_data/corners_only_2/fulltake_gid[1][0-6].pkl")
# files.extend(files_2)

data = []

for file in files:
    df = pd.read_pickle(file)
    df.x *= mu_factor
    df.y *= mu_factor
    data.append(df)

print(f"Loaded {len(data)} configurations.")
[
    print(
        f"\t - Grid {df.gid.iloc[0]}: {len(df)} total data points | {len(df.rid.unique())} runs | {len(df[df.rid==0])} data points / run"
    )
    for df in data
]
display(data[0])


# Debug

In [ ]:
reload(analysis)

global_x_v_a = []

for df_gid in data:

    # x, v, a values for this entire config
    grid_x_v_a = []

    # compute speed and acc for each run
    for rid, df_rid in df_gid.groupby("rid"):
        x_v_a = analysis.calc_v_a_from_position(df_rid.x, df_rid["time[hr]"])
        x_v_a[df_gid.iloc[0][5:].index] = df_gid.iloc[0][5:]
        grid_x_v_a.append(x_v_a)

    grid_x_v_a = pd.concat(grid_x_v_a)

    # with a 112mu mp, this gives bins of dim 3.5mu x 3.5mu
    nbins = 32
    analysis.get_bin_indices(grid_x_v_a, nbins)

    global_x_v_a.append(grid_x_v_a)


gid_to_indx = dict(
    (global_x_v_a[k].iloc[0].gid.astype(int), k) for k in range(len(global_x_v_a))
)

dt = 0.0015 * 250 * 8 / 60  # hr


In [ ]:
df_gid = global_x_v_a[gid_to_indx[gid]]

F, F_std, sigma = analysis.calc_F_sigma(df_gid, dt, nbins, min_pts=1)

bounds = df_gid.agg(["min", "max"])
xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]


plot_title = _make_title(df_gid)
mp_type = _get_mp_type(f"../configs/IM/grid_id{gid}/simbox.yaml")
plot_title += f"substrate = {mp_type}"
bounds = (xmin, xmax, vmin, vmax, nbins)
title = {"title": plot_title, "size": 20}

reload(analysis)

init_pts = linear_init_pts(bounds, n_pts=50, basin_only=True)
# init_pts = F_init_pts(F, bounds)[::5]
fig = analysis.imshow_F_sigma([F, sigma], bounds, init_pts, title, interp="bilinear")
# fig.axes[2].scatter(init_pts[:, 0], init_pts[:, 1])

# import pickle

# with open("fig.pkl", "wb") as f:
#     pickle.dump({"fig": fig}, f)


In [ ]:
import sys

sys.exit(1)


In [ ]:
import pickle
import os

os.system(
    "rm ../_server/sim_data/corners_only_2/single_run_long/superimpose\ single\ run\ onto\ phase\ space/*.png"
)

for k in range(10):
    with open("fig.pkl", "rb") as f:
        fig = pickle.load(f)["fig"]

    x, v = xv_traj[k * 100 : 100 * (k + 1), 0], xv_traj[k * 100 : 100 * (k + 1), 1]
    fig.axes[2].scatter(x, v, s=50, color="orange", zorder=2)
    fig.savefig(
        f"../_server/sim_data/corners_only_2/single_run_long/superimpose single run onto phase space/img_{k}.png"
    )
    plt.close()


In [ ]:
import pickle
import os

# os.system("rm ../_server/sim_data/corners_only_2/single_run_long/superimpose\ single\ run\ onto\ phase\ space/imposed_raw/*.png")
for k in range(362, 450):
    with open("fig.pkl", "rb") as f:
        fig = pickle.load(f)["fig"]

    x, v = xv_traj[k, 0], xv_traj[k, 1]
    fig.axes[2].scatter(x, v, s=50, color="orange", zorder=2)
    fig.savefig(
        f"../_server/sim_data/corners_only_2/single_run_long/superimpose single run onto phase space/imposed_raw/img_{k}.png"
    )
    plt.close()


In [ ]:
from PIL import Image
import os

os.system(
    "rm ../_server/sim_data/corners_only_2/single_run_long/superimpose\ single\ run\ onto\ phase\ space/combined/*"
)

i = 0
for k in range(0, 100):
    images = [
        Image.open(x)
        for x in [
            f"../_server/sim_data/corners_only_2/single_run_long/superimpose single run onto phase space/imposed_raw/img_{k}.png",
            f"../_server/sim_data/corners_only_2/single_run_long/superimpose single run onto phase space/raw/img_{k}.png",
        ]
    ]
    widths, heights = zip(*(i.size for i in images))

    x_offset = 0
    y_offset = 0
    total_width = sum(widths) + x_offset
    max_height = max(heights)
    new_im = Image.new("RGB", (total_width, max_height))
    for im in images:
        new_im.paste(im, (x_offset, y_offset))
        x_offset += im.size[0]
        y_offset += im.size[1] // 2

    new_im.save(
        f"../_server/sim_data/corners_only_2/single_run_long/superimpose single run onto phase space/combined/img_{i}.png"
    )
    i += 1


# Back to original code

#### Calculate $\{v, a\}$ and bin $\{x, v, a\}$
- `global_x_v_a`: `list`, with each element being a `pd.DataFrame` for an entire grid parameter. The data frame contains the position and speed for all the runs. 

In [ ]:
global_x_v_a = []

for df_gid in data:

    # x, v, a values for this entire config
    grid_x_v_a = []

    # compute speed and acc for each run
    for rid, df_rid in df_gid.groupby("rid"):
        x_v_a = analysis.calc_v_a_from_position(df_rid.x, df_rid["time[hr]"])
        x_v_a[df_gid.iloc[0][5:].index] = df_gid.iloc[0][5:]
        grid_x_v_a.append(x_v_a)

    grid_x_v_a = pd.concat(grid_x_v_a)

    # with a 112mu mp, this gives bins of dim 3.5mu x 3.5mu
    nbins = 32
    analysis.get_bin_indices(grid_x_v_a, nbins)

    global_x_v_a.append(grid_x_v_a)

global_x_v_a[0]


#### Acceleration maps and flow lines

- Consider each grid's behavior aggregated across all its runs.

In [ ]:
reload(analysis)

gid_to_indx = dict(
    (global_x_v_a[k].iloc[0].gid.astype(int), k) for k in range(len(global_x_v_a))
)

dt = 0.0015 * 250 * 8 / 60  # hr

# gids = np.arange(len(global_x_v_a))
gids = [1]
for gid in gids:
    df_gid = global_x_v_a[gid_to_indx[gid]]

    F, F_std, sigma = analysis.calc_F_sigma(df_gid, dt, nbins, min_pts=50)

    bounds = df_gid.agg(["min", "max"])
    xmin, xmax = bounds["x"]
    vmin, vmax = bounds["v"]

    plot_title = _make_title(df_gid)
    mp_type = _get_mp_type(f"../configs/IM/grid_id{gid}/simbox.yaml")
    plot_title += f"substrate = {mp_type}"
    bounds = (xmin, xmax, vmin, vmax, nbins)
    title = {"title": plot_title, "size": 20}

    xmin, xmax = 127, 172
    init_pts = linear_init_pts(xmin, xmax, vmin, vmax, n_pts=10, s=1, basin_only=False)
    # init_pts = F_init_pts(F, bounds)[::3]

    analysis.F_streamplot(
        F,
        bounds,
        stream_init_pts=init_pts,
        title=title,
        interp="bilinear",
        # save_path=f"../_server/sim_data/corners_only_2/grid_{gid}.png",
    )


# Misc.

#### Position dist

In [ ]:
reload(analysis)

plt.rc("axes", titlesize=8)
analysis.position_dist(data, "two_state", mu_factor)


#### Hopping times

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(8, 6), dpi=200)
plt.rc("axes", titlesize=12)

x, y = np.meshgrid([0, 1, 2], [0, 1, 2])

# in microns
d = 73 / 2
xL = (L_box / 2) * 6 - d
xR = (L_box / 2) * 6 + d
buffer = 38 / 2

for i, j, df in zip(x.flatten(), y.flatten(), data):
    beta = df.beta.iloc[0]
    gamma = df.gamma.iloc[0]
    D = df.D.iloc[0]
    axs[i, j].set_title(rf"$\beta$ = {beta}, $\gamma$ = {gamma}, $D$ = {D}", y=1.1)

    hop_times = []
    for rid, d in df.groupby("rid"):
        for t in analysis.get_hopping_times(d, xL, xR, buffer):
            hop_times.append(t)

    if len(hop_times) == 0:
        continue

    axs[i, j].hist(hop_times, color="red", alpha=0.5, density=True)
    axs[i, j].set_xlabel("Hopping time (hr)", fontsize=14)

plt.tight_layout()
plt.subplots_adjust(hspace=2, wspace=2)
plt.show()
